<a href="https://colab.research.google.com/github/Housenkan/ChallengeAluraTelecomX_Part_2/blob/main/TelecomX_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

dados = pd.read_csv('/content/dados_tratados.csv')